# tabarules

Meant to take tabular data and turn it into a format usable for `mlxtend` association rules mining.

In [1]:
#!pip install mlxtend

### functions to write
* ~~float_processor~~
* ~~boolean_processor~~ (needs to be robust to categorial)
* ~~int_processor (debatable if needed)~~
* ~~category_processor~~
* determine_dtype (takes series and identifies relevant fxn)
* featurize_df

__other associated todos__
* write tests to ensure that the data coming in is actually of the right type, throw error messages

In [2]:
# imports
import os
import numpy as np
import pandas as pd
import mlxtend
from math import ceil

In [3]:
# this is the nan functionality

sample = pd.Series([np.nan, 0, np.nan, 7])
np.where(np.isnan(sample))

(array([0, 2], dtype=int64),)

In [4]:
# load in data
df = pd.read_csv("./data/winequality-red-features.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,high_alc,acid_level,quality_2
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0,low,bad
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0,med,bad
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0,med,bad
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0,med,bad
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0,low,bad


In [5]:
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
high_alc                  int64
acid_level               object
quality_2                object
dtype: object

So the idea here is that we want to make each row of our table a list of values, and have each of these stored with an index. That's the format that `mlxtend` expects for apriori and association rules.

In [6]:
float_series = df['fixed acidity']
categorical_series = df['acid_level']
boolean_series = df['quality_2']
boolean_series2 = df['high_alc']
int_series = df['quality']

In [7]:
def float_processer(series, \
                    title, \
                    cutoffs = 4, \
                    na_action = ['label', 'return_na'][1], \
                    print_labels = False):
    '''
    SUMMARY: Gives bin labeling to float data based off of splits in the data
    
    INPUTS:
    - series: A pandas series object of type float
    - title: A title used for the feature of type string
    - cutoffs: the number of bins to place data into. The more bins, the more granular.
    - na_action: desired action for missing values. "label" will return a string with "title_is_na", "return_na" will return NA
    - print_labels: a boolean value to print out the label values before assignment. Useful for debugging.
    
    OUTPUTS:
    - a pandas series object of the bin-labelled data
    
    NOTES:
    - string labels can get long. opt for shorter ones if possible.
    - string labels will come out in the format "title_lessthan_cutoffval" or "title_morethan_cutoffval"
    - Cutoff values are calulated by sorting the data and taking the value at each index 1/cutoffs way through the data.
    
    '''
    
    
    # find splits in data based on ordering
    not_nas = np.where(np.logical_not(np.isnan(series)))
    not_na_series = series[not_nas[0]]
    len_series = not_na_series.shape[0]
    len_tot_series = series.shape[0]
    series_sorted = not_na_series.sort_values().reset_index(drop = True)
    cutoff_val_ind = [ceil( (i+1) / cutoffs * len_series) for i in range(cutoffs-1)]
    cutoff_vals = series_sorted[cutoff_val_ind].values
    
    # label creation
    str_labels = [title + "_lessthan_" + str(val) for val in cutoff_vals]
    str_labels.append(title + "_morethan_" + str(cutoff_vals[cutoffs-2]))
    
    if print_labels:
        print(str_labels)
    
    output_list = []
    # likely can have better parallelization if needed
    # populate a list with the appropriate label for the data
    for i in range(len_tot_series):
        if pd.isna(series[i]):
            if na_action == "label":
                output_list.append(title + "_is_na")
                continue
            elif na_action == "return_na":
                output_list.append(np.nan)
                continue
        for j in range(len(cutoff_vals)):
            if series[i] <= cutoff_vals[j]:
                output_list.append(str_labels[j])
                break
            elif j == len(cutoff_vals)-1:
                output_list.append(str_labels[len(cutoff_vals)])
                break
    
    # can debate whether list or series is better later
    return pd.Series(output_list)


In [8]:
# test for float_processor

print(float_processer(float_series, "fixed_acidity", 4, print_labels = True)[0:20])
print(float_series[0:20])

['fixed_acidity_lessthan_7.1', 'fixed_acidity_lessthan_7.9', 'fixed_acidity_lessthan_9.2', 'fixed_acidity_morethan_9.2']
0     fixed_acidity_lessthan_7.9
1     fixed_acidity_lessthan_7.9
2     fixed_acidity_lessthan_7.9
3     fixed_acidity_morethan_9.2
4     fixed_acidity_lessthan_7.9
5     fixed_acidity_lessthan_7.9
6     fixed_acidity_lessthan_7.9
7     fixed_acidity_lessthan_7.9
8     fixed_acidity_lessthan_7.9
9     fixed_acidity_lessthan_7.9
10    fixed_acidity_lessthan_7.1
11    fixed_acidity_lessthan_7.9
12    fixed_acidity_lessthan_7.1
13    fixed_acidity_lessthan_7.9
14    fixed_acidity_lessthan_9.2
15    fixed_acidity_lessthan_9.2
16    fixed_acidity_lessthan_9.2
17    fixed_acidity_lessthan_9.2
18    fixed_acidity_lessthan_7.9
19    fixed_acidity_lessthan_7.9
dtype: object
0      7.4
1      7.8
2      7.8
3     11.2
4      7.4
5      7.4
6      7.9
7      7.3
8      7.8
9      7.5
10     6.7
11     7.5
12     5.6
13     7.8
14     8.9
15     8.9
16     8.5
17     8.1
18     

In [9]:
def boolean_processer(series, title, which_yes = 1, na_action = ['label', 'return_na'][1], print_labels = False):
    '''
    SUMMARY: Gives bin labeling to boolean data
    
    INPUTS:
    - series: A pandas series object of type float
    - title: A title used for the feature of type string
    - which_yes: which number in the data is used as the "yes" marking
    - print_labels: a boolean value to print out the label values before assignment. Useful for debugging.
    
    OUTPUTS:
    - a pandas series object of the bin-labelled data
    
    NOTES:
    - string labels can get long. opt for shorter ones if possible.
    - string labels will come out in the format "title_yes" or "title_no"
    - Cutoff values are calulated by sorting the data and taking the value at each index 1/cutoffs way through the data.
    
    '''
    len_series = series.shape[0]
    
    # label creation
    str_labels = [title + "_no", title + "_yes"]
    
    if print_labels:
        print(str_labels)
    
    output_list = []
    # likely can have better parallelization if needed
    # populate a list with the appropriate label for the data
    
    ### 
    for i in range(len_series):
        # deal with nas first
        if pd.isna(series[i]):
            if na_action == "label":
                output_list.append(title + "_is_na")
                continue
            elif na_action == "return_na":
                output_list.append(np.nan)
                continue
        # logic for dealing with regular values
        if series[i] == which_yes:
            output_list.append(str_labels[1])
        else:
            output_list.append(str_labels[0])
    
    # can debate whether list or series is better later
    return pd.Series(output_list)


In [10]:
# works
boolean_processer(boolean_series2, "high_alc")

0        high_alc_no
1        high_alc_no
2        high_alc_no
3        high_alc_no
4        high_alc_no
5        high_alc_no
6        high_alc_no
7        high_alc_no
8        high_alc_no
9       high_alc_yes
10       high_alc_no
11      high_alc_yes
12       high_alc_no
13       high_alc_no
14       high_alc_no
15       high_alc_no
16      high_alc_yes
17       high_alc_no
18       high_alc_no
19       high_alc_no
20       high_alc_no
21       high_alc_no
22       high_alc_no
23       high_alc_no
24       high_alc_no
25       high_alc_no
26       high_alc_no
27       high_alc_no
28       high_alc_no
29       high_alc_no
            ...     
1569    high_alc_yes
1570    high_alc_yes
1571    high_alc_yes
1572     high_alc_no
1573    high_alc_yes
1574    high_alc_yes
1575    high_alc_yes
1576    high_alc_yes
1577    high_alc_yes
1578    high_alc_yes
1579    high_alc_yes
1580    high_alc_yes
1581    high_alc_yes
1582    high_alc_yes
1583     high_alc_no
1584    high_alc_yes
1585    high_

In [11]:
boolean_processer(boolean_series, "quality", which_yes = "good")

0        quality_no
1        quality_no
2        quality_no
3        quality_no
4        quality_no
5        quality_no
6        quality_no
7       quality_yes
8       quality_yes
9        quality_no
10       quality_no
11       quality_no
12       quality_no
13       quality_no
14       quality_no
15       quality_no
16      quality_yes
17       quality_no
18       quality_no
19       quality_no
20       quality_no
21       quality_no
22       quality_no
23       quality_no
24       quality_no
25       quality_no
26       quality_no
27       quality_no
28       quality_no
29       quality_no
           ...     
1569     quality_no
1570     quality_no
1571     quality_no
1572     quality_no
1573     quality_no
1574     quality_no
1575     quality_no
1576     quality_no
1577     quality_no
1578     quality_no
1579     quality_no
1580     quality_no
1581     quality_no
1582     quality_no
1583     quality_no
1584    quality_yes
1585     quality_no
1586     quality_no
1587     quality_no


In [12]:
float_processer(int_series, "quality", cutoffs = 7, print_labels = True).unique()

['quality_lessthan_5', 'quality_lessthan_5', 'quality_lessthan_5', 'quality_lessthan_6', 'quality_lessthan_6', 'quality_lessthan_6', 'quality_morethan_6']


array(['quality_lessthan_5', 'quality_lessthan_6', 'quality_morethan_6'],
      dtype=object)

In [13]:
int_series.unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [14]:
def cat_processer(series, title, na_action = ['label', 'return_na'][1], print_labels = False):
    '''
    SUMMARY: Gives bin labeling to categorical data based off of splits in the data
    
    INPUTS:
    - series: A pandas series object of type float
    - title: A title used for the feature of type string
    - print_labels: a boolean value to print out the label values before assignment. Useful for debugging.
    
    OUTPUTS:
    - a pandas series object of the bin-labelled data
    
    NOTES:
    - string labels can get long. opt for shorter ones if possible.
    - string labels will come out in the format "title_is_label"
    
    '''
    
    
    # find splits in data based on ordering
    len_series = series.shape[0]
    not_nas = np.where(np.logical_not(pd.isna(series)))
    not_na_series = series[not_nas[0]]
    unique_vals = not_na_series.unique()
    
    # label creation
    str_labels = [title + "_is_" + val for val in unique_vals]
    
    if print_labels:
        print(str_labels)
    
    output_list = []
    # likely can have better parallelization if needed
    # populate a list with the appropriate label for the data
    for i in range(len_series):
        # deal with nas first
        if pd.isna(series[i]):
            if na_action == "label":
                output_list.append(title + "_is_na")
                continue
            elif na_action == "return_na":
                output_list.append(np.nan)
                continue
        for j in range(unique_vals.shape[0]):
            if series[i] == unique_vals[j]:
                output_list.append(str_labels[j])
                next
    
    # can debate whether list or series is better later
    return pd.Series(output_list)


In [15]:
cat_processer(categorical_series, "acid_level", True)

0       acid_level_is_low
1       acid_level_is_med
2       acid_level_is_med
3       acid_level_is_med
4       acid_level_is_low
5       acid_level_is_low
6       acid_level_is_med
7       acid_level_is_med
8       acid_level_is_med
9       acid_level_is_med
10      acid_level_is_med
11      acid_level_is_med
12      acid_level_is_low
13      acid_level_is_med
14      acid_level_is_med
15      acid_level_is_med
16      acid_level_is_med
17      acid_level_is_med
18      acid_level_is_med
19      acid_level_is_med
20      acid_level_is_med
21      acid_level_is_low
22      acid_level_is_med
23      acid_level_is_med
24      acid_level_is_med
25      acid_level_is_med
26      acid_level_is_med
27      acid_level_is_med
28      acid_level_is_med
29      acid_level_is_med
              ...        
1569    acid_level_is_med
1570    acid_level_is_med
1571    acid_level_is_med
1572    acid_level_is_med
1573    acid_level_is_low
1574    acid_level_is_med
1575    acid_level_is_med
1576    acid

In [16]:
cat_processer(boolean_series, "quality_2")

0        quality_2_is_bad
1        quality_2_is_bad
2        quality_2_is_bad
3        quality_2_is_bad
4        quality_2_is_bad
5        quality_2_is_bad
6        quality_2_is_bad
7       quality_2_is_good
8       quality_2_is_good
9        quality_2_is_bad
10       quality_2_is_bad
11       quality_2_is_bad
12       quality_2_is_bad
13       quality_2_is_bad
14       quality_2_is_bad
15       quality_2_is_bad
16      quality_2_is_good
17       quality_2_is_bad
18       quality_2_is_bad
19       quality_2_is_bad
20       quality_2_is_bad
21       quality_2_is_bad
22       quality_2_is_bad
23       quality_2_is_bad
24       quality_2_is_bad
25       quality_2_is_bad
26       quality_2_is_bad
27       quality_2_is_bad
28       quality_2_is_bad
29       quality_2_is_bad
              ...        
1569     quality_2_is_bad
1570     quality_2_is_bad
1571     quality_2_is_bad
1572     quality_2_is_bad
1573     quality_2_is_bad
1574     quality_2_is_bad
1575     quality_2_is_bad
1576     qua

In [17]:
nan_series_float = pd.Series([1, 4.2, np.nan, 5, 12, 1, 1.2, 7, np.nan])

In [18]:
float_processer(nan_series_float, "tester", cutoffs = 4, print_labels = True, na_action = 'label')

['tester_lessthan_1.2', 'tester_lessthan_5.0', 'tester_lessthan_12.0', 'tester_morethan_12.0']


0     tester_lessthan_1.2
1     tester_lessthan_5.0
2            tester_is_na
3     tester_lessthan_5.0
4    tester_lessthan_12.0
5     tester_lessthan_1.2
6     tester_lessthan_1.2
7    tester_lessthan_12.0
8            tester_is_na
dtype: object

In [19]:
nan_series_bool = pd.Series([0,1,np.nan, 1, 0, 1, 1, np.nan, 0, np.nan])
boolean_processer(nan_series_bool, title = "tester", na_action = 'label')

0       tester_no
1      tester_yes
2    tester_is_na
3      tester_yes
4       tester_no
5      tester_yes
6      tester_yes
7    tester_is_na
8       tester_no
9    tester_is_na
dtype: object

In [20]:
nan_series_strings = pd.Series(['aaa', np.nan, 'bbb', np.nan, 'aaa', 'ccc'])
cat_processer(nan_series_strings, title = 'tester', na_action = 'label')

0    tester_is_aaa
1     tester_is_na
2    tester_is_bbb
3     tester_is_na
4    tester_is_aaa
5    tester_is_ccc
dtype: object

In [21]:
float_processer(int_series, title = "tester", print_labels = True)

['tester_lessthan_5', 'tester_lessthan_6', 'tester_lessthan_6', 'tester_morethan_6']


0       tester_lessthan_5
1       tester_lessthan_5
2       tester_lessthan_5
3       tester_lessthan_6
4       tester_lessthan_5
5       tester_lessthan_5
6       tester_lessthan_5
7       tester_morethan_6
8       tester_morethan_6
9       tester_lessthan_5
10      tester_lessthan_5
11      tester_lessthan_5
12      tester_lessthan_5
13      tester_lessthan_5
14      tester_lessthan_5
15      tester_lessthan_5
16      tester_morethan_6
17      tester_lessthan_5
18      tester_lessthan_5
19      tester_lessthan_6
20      tester_lessthan_6
21      tester_lessthan_5
22      tester_lessthan_5
23      tester_lessthan_5
24      tester_lessthan_6
25      tester_lessthan_5
26      tester_lessthan_5
27      tester_lessthan_5
28      tester_lessthan_5
29      tester_lessthan_6
              ...        
1569    tester_lessthan_6
1570    tester_lessthan_6
1571    tester_lessthan_6
1572    tester_lessthan_5
1573    tester_lessthan_6
1574    tester_lessthan_6
1575    tester_lessthan_6
1576    test

In [22]:
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
high_alc                  int64
acid_level               object
quality_2                object
dtype: object

In [23]:
df.dtypes.index

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'high_alc', 'acid_level',
       'quality_2'],
      dtype='object')

In [82]:
# function to take entire dataframe and process into listed strings
# todo: customizable for non-defaults
def list_featurize_df(df, \
                      index_list = [i for i in range(df.shape[0])]):
    # loop thru series
    df_out = pd.DataFrame()
    for column in df.columns:
        out_series = list_featurize_series(df[column], title = column)
        df_out[column] = out_series

    kv_store = {}
    # upon completion, go thru and make a key-value store
    for i in range(df.shape[0]):
        writing_vals = df_out.loc[i].values[~pd.isna(df_out.loc[i].values)]
        kv_store[index_list[i]] = [val for val in writing_vals]
    
    return(kv_store)
    # then iterate thru and make row-wise key value store
    

In [83]:
# function to take individual series and impose the right function on it
def list_featurize_series(series,\
                          title, \
                          cutoffs = 4, \
                          which_yes = 1, \
                          na_action = ['label', 'return_na'][1], \
                          print_labels = False):
    # first figure out what type the series is
    typeof = series.dtypes
    
    # categorical
    if typeof == 'O':
        ret_series = cat_processer(series, \
                                   title, \
                                   na_action, \
                                   print_labels)    

    # only boolean
    elif np.array_equal(series.unique()[~pd.isna(series.unique())],\
                        np.array([0,1])):
        
        ret_series = boolean_processer(series, \
                                       title, \
                                       which_yes, \
                                       na_action, \
                                       print_labels)
    
    # floats/ints
    else:
        ret_series = float_processer(series, \
                    title, \
                    cutoffs, \
                    na_action, \
                    print_labels)
    
    return ret_series

In [84]:
boolean_series.dtypes

dtype('O')

In [85]:
boolean_series2.unique()

array([0, 1], dtype=int64)

In [86]:
np.array_equal(nan_series_bool.unique()[~np.isnan(nan_series_bool.unique())], np.array([0,1]))

True

In [87]:
np.array([0,1,np.nan])

array([ 0.,  1., nan])

In [88]:
nan_series_bool.unique()

array([ 0.,  1., nan])

In [89]:
list_featurize_series(nan_series_strings, title = "testint")

0    testint_is_aaa
1               NaN
2    testint_is_bbb
3               NaN
4    testint_is_aaa
5    testint_is_ccc
dtype: object

In [90]:
df2 = pd.DataFrame()

In [91]:
df2["col1"] = nan_series_bool

In [92]:
df2

,col1
0,0.0
1,1.0
2,NaN
3,1.0
4,0.0
5,1.0
6,1.0
7,NaN
8,0.0
9,NaN


In [93]:
df2["col3"] = df2['col1'] + 1

In [94]:
df2

,col1,col3
0,0.0,1.0
1,1.0,2.0
2,NaN,NaN
3,1.0,2.0
4,0.0,1.0
5,1.0,2.0
6,1.0,2.0
7,NaN,NaN
8,0.0,1.0
9,NaN,NaN


In [95]:
list_featurize_df(df2)

{0: ['col1_no', 'col3_lessthan_1.0'],
 1: ['col1_yes', 'col3_lessthan_2.0'],
 2: [],
 3: ['col1_yes', 'col3_lessthan_2.0'],
 4: ['col1_no', 'col3_lessthan_1.0'],
 5: ['col1_yes', 'col3_lessthan_2.0'],
 6: ['col1_yes', 'col3_lessthan_2.0'],
 7: [],
 8: ['col1_no', 'col3_lessthan_1.0'],
 9: []}

In [96]:
list_featurize_df(df)

{0: ['fixed acidity_lessthan_7.9',
  'volatile acidity_morethan_0.64',
  'citric acid_lessthan_0.09',
  'residual sugar_lessthan_1.9',
  'chlorides_lessthan_0.079',
  'free sulfur dioxide_lessthan_14.0',
  'total sulfur dioxide_lessthan_38.0',
  'density_lessthan_0.99785',
  'pH_morethan_3.4',
  'sulphates_lessthan_0.62',
  'alcohol_lessthan_9.5',
  'quality_lessthan_5',
  'high_alc_no',
  'acid_level_is_low',
  'quality_2_is_bad'],
 1: ['fixed acidity_lessthan_7.9',
  'volatile acidity_morethan_0.64',
  'citric acid_lessthan_0.09',
  'residual sugar_lessthan_2.6',
  'chlorides_morethan_0.09',
  'free sulfur dioxide_morethan_21.0',
  'total sulfur dioxide_morethan_62.0',
  'density_lessthan_0.99785',
  'pH_lessthan_3.21',
  'sulphates_lessthan_0.73',
  'alcohol_lessthan_10.2',
  'quality_lessthan_5',
  'high_alc_no',
  'acid_level_is_med',
  'quality_2_is_bad'],
 2: ['fixed acidity_lessthan_7.9',
  'volatile acidity_morethan_0.64',
  'citric acid_lessthan_0.09',
  'residual sugar_lesst